Python is actually quite slow for dong pure math.  We covered some of the reasons when discussing Numpy.  Numpy addresses a lot of these, but it only really works when your can express your problems in the language of arrays, vectorization, and things Numpy has built-in.  Sometimes--many times--this is all you need, but sometimes you need to go further.  This notebook shows a few tools for getting even more speed out of your code, especially when it does a lot of math, and has a lot of very "tight" loops that need to run a whole lot of times.

# Numba: Speedy speedy math

As fast as Numpy is for doing math, it's still not as fast as we could get.  The fastest possible code would be *compiled* ahead of time down to machine code.  This requires a *compiler*: a program that reads the code we write, "translates" it into the 1s and 0s that the computer understands, and usually performs a lot of extra optimizations along the way to get us faster code with the same outputs.  Unfortunately, Python is an interpreted language--it gets executed one line at a time, with very little optimization--which makes writing it easier in some cases, but at the cost of speed.  Even Numpy is still interpreted.  Numpy is using a lot of compiled C code--which is very fast--but there's still Python interpretation that needs to happen.

Enter Numba: a library that will compile your functions for you, using something called *just in time* (or JIT) compilation.  The "just in time" part isn't something we need to worry about, though.  Basically, Numba gives you free extra speed.  Sometimes a *lot*, with some tweaks to your code.

Install Numba with:

```bash
conda install numba
```

Note: the currently available versions of Numba require a Numpy version no later than 1.20; the current Numpy versions you can get through Conda are generally newer.  So you may need to use the command:

```bash
conda install numba numpy<=1.20
```

Then import the `jit` and `njit` decorators.  `njit` is generally faster, but doesn't work on as much Python code (it requires things to be more directly translatable to simpler languages, but it can generate much faster code if that's the case).  `jit` is more flexible, but not always as fast.

`jit`/`njit` are what are called *decorators.*  This is a piece of pretty complex Python concept that I've intentionally not covered.  It's closely related to the idea of a *closure* in general programming/computer science, but for our purposes, we're going to treat it like a little bit of magic and not worry about what's going on.  Basically: we write `@jit`/`@njit` on the line right before the function we want to speed up.  There are a few arguments we can pass to the decorators, e.g., `@njit(nogil=True)`.  Not having the parentheses uses all the default settings.

In [1]:
from numba import jit, njit
import numpy as np
from timeit import timeit

def dot_product(x, y):
    total = 0
    for i in range(len(x)):
        total = total + x[i] * y[i]
    return total

# can also ust @njit here--this code doesn't benefit much from it
@jit
def dot_product_numba(x, y):
    total = 0
    for i in range(len(x)):
        total = total + x[i] * y[i]
    return total

rng = np.random.default_rng()
x = rng.random(size=1000)
y = rng.random(size=1000)

print(
    "Dot product, 10,000 times, normal Python:",
    timeit("dot_product(x, y)", globals=globals(), number=10000)
)

print(
    "Dot product, 10,000 times, Numba acceleration:",
    timeit("dot_product_numba(x, y)", globals=globals(), number=10000)
)

Dot product, 10,000 times, normal Python: 5.1984004
Dot product, 10,000 times, Numba acceleration: 0.9420255999999991


In the above example, `np.dot()` would actually faster for this specific use case--but just adding one decorator to your code gets you a free speedup!

Numba has a lot of depth to it--options for GPU acceleration, using either NVidia or AMD GPUs; very fast, low-level parallelization; and more.  Using all those tricks can actually get your Python code *almost as fast as really good C code*--at the expense of some non-trivial re-writes to your program.  

# Cupy: Numpy on your GPU

If you have an NVidia GPU, you can install the `cupy` library, which gives you Numpy arrays, but on your GPU.  This has some tradeoffs compared to normal Numpy:

- Cupy is generally a drop-in replacement for Numpy *in your code*--they try very hard to re-implement as much of the Numpy API as possible, so that it magically "just works" on the GPU.  If you try to pass a Cupy array to a library that expects Numpy arrays, it might throw an error.
    - Some changes to recent versions of Numpy are *incompatible* with Cupy, e.g., using `np.random.default_rng()`.
    - Some functions' positional arguments may also be different from Numpy, but passing function arguments by name usually solves this.
- Cupy will generally be faster than Numpy for the same operations, especially massively parallelizeable ones like matrix multiplication.
- Because Cupy is running on your GPU, you will be limited by *your GPU's available RAM*, which will almost always be less than your main system RAM.
    - You can get computers/servers/cloud instances with hundreds of gigabytes of RAM.  It might cost a few hundred dollars (or more), but realistically, a few hundred dollars of main system RAM costs about as much as one or two decent GPUs.  Each of those GPUs might have 12-16gb of RAM.  So if you happen to have really, really big arrays, you probably need to use Numpy.
- Cupy is really designed for NVidia GPUs, and may not run well on AMD GPUs.  I don't think it'll run at all on Intel graphics.
    - There is [expermental support for AMD GPUs](https://docs.cupy.dev/en/v11.0.0b2/install.html#using-cupy-on-amd-gpu-experimental), but since it's only experimental, it may be prone to crashes/bugs/performance issues at this point in time.  (I don't have access to an AMD GPU to test with).
- As is generally true for GPU programming, getting data to and from the GPU is the hardest part.  Generally, you'll want to move as many things to the GPU (i.e., into Cupy arrays) as you can, do all your array logic there, and then move them back.  Try to make sure you're only operating on Cupy arrays using other Cupy arrays to avoid the CPU-to-GPU data transfer bottleneck.

GPUs excel at *matrix multiplication* and *dot products of very large arrays*--so if you happen to be doing matrix multiplication, expect *huge* speedups.  A lot of other array operations are faster on GPUs too, but matrix multiplication is the most extreme and noteworthy example.

Installing Cupy requires installing the `cudatoolkit` package through conda.  This package is required to run general-purpose compute on your NVidia GPU, and it will require that you have a current NVidia driver installed *system-wide* (you can't install NVidia drivers through conda--or at least, you can't do that and expect them to work well).

```bash
conda install cudatoolkit cupy
```

From there, just replace Numpy with Cupy in your code.

In [2]:
from timeit import timeit
import cupy as cp
# cupy does not support numpy's `np.random.rng()` interface.
# it does follow Numpy's older--and still supported--
# `np.random` interface.
cp_x = cp.random.random(size=(1000, 1000))
print(
    "Cupy, 1000x1000 matrix multiplication, 100 times: ",
    timeit("y = cp_x @ cp_x", globals=globals(), number=100)
)

import numpy as np
np_rng = np.random.default_rng()
np_x = np_rng.random(size=(1000, 1000))
print(
    "Numpy, 1000x1000 matrix multiplication, 100 times: ",
    timeit("y = np_x @ np_x", globals=globals(), number=100)
)

C:\Users\andersonh\Miniconda3\envs\PythonForDataAnalysis\lib\site-packages\cupy\_environment.py:205: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


Cupy, 1000x1000 matrix multiplication, 100 times:  0.5187214999999998
Numpy, 1000x1000 matrix multiplication, 100 times:  182.43337879999999


Dang.  That's a nice speedup.

# Cython: C-like speed, Python-like syntax

We're not even going to scratch the surface here--I'm just going to put the name "Cython" on your radar.

Numba is great for general-purpose speedup, but it's still limited by the fact that it has to work with Python code.  Cython exists partly to solve that problem.

Strictly speaking, Cython is actually *a different programming language.*  It's a *proper superset* of Python: all Python code is valid Cython code, but not vice versa.  Cython's biggest additions are the following:
- Static typing.  This bypasses a lot of Python's slowness, since a lot of that comes from the runtime type checking.
- Ability to call functions from C libraries, which will give huge speedups.
- Ability to *transpile* code from Python to C, then compile that C code into a very fast binary file that you can load like a Python library.
    - Cython code, once compiled like this, has to be imported into another Python program; it can't really be run directly.

There is a *lot* to Cython, since it's a whole programming language.  There is an extremely good chance you will never need it; you can probably get all the speed you need with Numpy, Numba, and maybe Cupy.  A lot of libraries we'll be seeing later (like Pandas and scikit-learn) have a lot of performance-critical parts written in Cython.  It's mostly used for when a *library* needs to be very fast, and it's not as commonly used in user-written code, since it's a good bit harder to write than plain Python.

It's good to know about, though, since there may come a day when you want to play with it, or when you end up needing to use it to write a very fast, efficient library.

(But, these days, if you need to write Python-like code that's very, very fast, it's becoming increasingly likely that you'll be switching to the Julia programming language).